In [ ]:
import numpy as np
import time

from qiskit.algorithms.optimizers import SPSA, COBYLA, SLSQP
from qiskit import QuantumCircuit
from qiskit.circuit import ParameterVector
from sklearn.datasets import make_moons

from activity_1_figure_utils import data_figure, history_figure, circuits_to_statevectors, bloch_sphere_statevector_figure, classification_figure
from activity_1_utils import prepare_all_circuits, train_classifier, classify, spsa_optimizer_callback, eval_cost_fct_quadratic
from activity_1_circuits import build_rotation_model_circuit, build_linear_model_circuit, build_layered_model_circuit

In [ ]:
from qiskit import Aer
from qiskit.utils import QuantumInstance
qasm_simulator = Aer.get_backend('qasm_simulator')
statevector_simulator = Aer.get_backend('statevector_simulator')
qasm_quantum_instance = QuantumInstance(qasm_simulator, shots = 1000)
sv_quantum_instance = QuantumInstance(statevector_simulator)

In [ ]:
n_samples = 100
data_xs, data_ys = make_moons(n_samples = n_samples,noise=0.1, random_state=0)

In [ ]:
fig = data_figure(data_xs,data_ys)
fig.show()

In [ ]:
# model = 'rotation'
# model = 'linear'
model = 'layered'
if model == 'rotation':
    classifier_circuit, data_params, model_params = build_rotation_model_circuit()
    initial_point = [0,0]
elif model == 'linear':
    classifier_circuit, data_params, model_params = build_linear_model_circuit()
    initial_point = [0,0,1,1]
elif model == 'layered':
    n_layers = 4
    classifier_circuit, data_params, model_params = build_layered_model_circuit(n_layers)
    initial_point = [0,0] * n_layers + [1,1] * n_layers

classifier_circuit.draw('mpl',scale = 2)

In [ ]:
t0 = time.time()

train_history = []
optimizer = SPSA(maxiter=50, callback=lambda n, p, v, ss, sa: spsa_optimizer_callback(n, p, v, ss, sa, train_history))

model_values, loss, nfev = train_classifier(optimizer,eval_cost_fct_quadratic,sv_quantum_instance,classifier_circuit,data_params,model_params,data_xs,data_ys,initial_point)

fig = history_figure(train_history)
fig.show()
print(model_values)
print(f'{time.time() - t0:.2f} sec')

In [ ]:
all_circuits = prepare_all_circuits(classifier_circuit,data_params,model_params,data_xs,model_values,add_measurements=False)

statevectors = circuits_to_statevectors(all_circuits)
fig = bloch_sphere_statevector_figure(statevectors,data_ys)
fig.show()

In [ ]:
predictions_ys = classify(sv_quantum_instance,classifier_circuit,model_params,model_values,data_params,data_xs)
fig = classification_figure(data_xs,data_ys,predictions_ys)
fig.show()

# Notes
Created by Maxime Dion <maxime.dion@usherbrooke.ca>